Conexión con BD OVSAI

In [2]:
! pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/14.3 MB 2.8 MB/s eta 0:00:05
   --------- ------------------------------ 3.4/14.3 MB 8.4 MB/s eta 0:00:02
   -------------------- ------------------- 7.3/14.3 MB 11.9 MB/s eta 0:00:01
   -------------------------------- ------- 11.5/14.3 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------  14.2/14.3 MB 15.3 MB/s eta 0:00:01
   ---------------------------------------- 14.3/14.3 MB 11.9 MB/s eta 0:00:00


In [1]:
from pysnmp.hlapi import *
from datetime import datetime
import pandas as pd
import mysql.connector
import time

In [2]:
OID_list = ['.1.3.6.1.4.1.29956.3.1.1.1.1.1.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.2.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.3.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.4.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.5.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.6.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.7.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.8.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.9.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.10.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.11.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.12.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.13.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.14.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.15.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.16.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.17.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.18.1',
    '.1.3.6.1.4.1.29956.3.1.1.1.1.19.1']

In [9]:
def getSnmp(target, community, port, OID_list, id_dispositivo):

    reading = []
    currentDateAndTime = datetime.now()
    date = str(pd.to_datetime(currentDateAndTime))
    reading.append(date) # primero se agrega la fecha

    # Realizar una operación SNMP GET
    for oid in OID_list:
        errorIndication, errorStatus, errorIndex, varBinds = next(
        getCmd(SnmpEngine(),
            CommunityData(community),
            UdpTransportTarget((target, port)),
            ContextData(),
            ObjectType(ObjectIdentity(oid)))
        )
        # Procesar la respuesta del GET
        if errorIndication:
            print(f"Error: {errorIndication}")
        elif errorStatus:
            print(f"Error en la petición: {errorStatus.prettyPrint()}")
        else:
            for varBind in varBinds:
                reading.append(varBind[1].prettyPrint())
                
    reading.append(id_dispositivo)
    
    return reading

In [10]:

# Conectar a la base de datos
conexion = mysql.connector.connect(
    host="localhost",         # Dirección del servidor MySQL
    user="root",        # Nombre de usuario de MySQL
    password="", # Contraseña del usuario de MySQL
    database="ovsai"          # Nombre de la base de datos
)

# Crear un cursor
cursor = conexion.cursor()

# Escribir la consulta SQL
consulta = "SELECT * FROM snmp"                      
insercion = "INSERT INTO oids_snmp (date, signalsnmp, noise, supply_voltage, rx_rate, reflected_power, temperature, rangesnmp, tx_rate, sn_delta, vendor, connected_to, upstream_signal, upstream_noise, disconnect_count, packet_rx_count, packet_tx_count, packet_dropped_count, packet_bad_count, id_dispositivo) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

while True:

    # Ejecutar la consulta
    cursor.execute(consulta)

    # Obtener todos los resultados
    resultados = cursor.fetchall()

    # Procesar los resultados
    for fila in resultados:
        
        # consulta a base de datos por las ips
        ip = fila[0]  # Dirección IP del dispositivo SNMP
        community = fila[1]  # Comunidad SNMP (debe coincidir con la configuración del dispositivo)
        port = fila[2]  # Puerto SNMP por defecto
        id_dispositivo = fila[3]
        
        # Se obtienen los valores snmp
        snmp = getSnmp(ip, community, port, OID_list, id_dispositivo)
        snmp.pop(1) # esta posicion hay basura por eso es necesario quitarla ('No Such Object currently exists at this OID')

        # Insercion en base datos de los snmp
        cursor.execute(insercion, (snmp))
        
    # Confirmar los cambios
    conexion.commit()
    
    time.sleep(60)  # Pausa el programa por 900 segundos (15 minutos)

    
# Cerrar el cursor y la conexión
cursor.close()
conexion.close()

KeyboardInterrupt: 